In [ ]:
import json

from src.aou_summary import *

hpo_of_interest = ''

start_date = '2018-02-11'
end_date = '2018-03-14'
interval = 'day' # One of: day, week, month

# start_date = '2017-05-21'
# end_date = '2018-03-10'
# interval = 'week' # One of: day, week, month

data_dir = '../../data/'
raw_data_path = data_dir + 'IntervalTotals/20180314_dates_all.txt'

summary = AouSummary(raw_data_path, start_date, end_date, interval)
csv_interval_totals = summary.csv_interval_totals

print('Full participants by site per ' + interval)
print('Date range: ' + start_date + ' - ' + end_date)
print('')
if hpo_of_interest != '':
    print('Daily counts for ' + hpo_of_interest + ':')    
print('\n'.join(csv_interval_totals))

if hpo_of_interest != '':
    print('')
    print('Total for ' + hpo_of_interest + ':')
    print(totals[hpo_of_interest])